1. 문서의 내용을 읽는다 (docx2txt)
2. 문서를 쪼갠다 (langchain-text-splitters)
  - 토큰수 초과로 답변을 생성하지 못할수 있고
  - 문서가 길면 (인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장 (OpenAIEmbeddings, chroma)
4. 질문이 있을 때, 백터 테이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 (similarity_search)

In [ ]:
%pip install --upgrade --quiet  docx2txt langchain-community -qU langchain-text-splitters -U langchain langchainhub --quiet

In [28]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1500, # 하나의 청크가 가질수있는 토큰수
  chunk_overlap=200 # 청크 간 중복 토큰수
)

loader = Docx2txtLoader("./tax_with_markdown.docx")

document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list[43]

In [42]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# 환경변수를 불러옴
load_dotenv()

# OpenAI에서 제공하는 Embedding Model을 활용해서 `chunk`를 vector화
embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-markdown-index' 
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [ ]:
%pip install langchain-chroma

In [ ]:
%pip install -qU langchain langchain-pinecone langchain-openai

In [85]:
query = '연봉 5,000만원인 직장인의 종합 소득세는?'

In [52]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')


In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [ ]:
retriever=database.as_retriever(search_kwargs={'k': 4})

retriever.invoke(query)

In [55]:
from langchain.chains import RetrievalQA

retriever=database.as_retriever()

qa_chain = RetrievalQA.from_chain_type( 
  llm,
  retriever=retriever,
  chain_type_kwargs={"prompt": prompt}
)
# query -> 직장인 -> 거주자 변경 chain 추가
ai_message = qa_chain({"query": query})

In [ ]:
ai_message

In [70]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

dictionary = ['사람을 나타내는 표현 -> 거주자']

prompt = ChatPromptTemplate.from_template(f"""
  사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
  만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않아도 됩니다.
  그런 경우에는 질문만 리턴해주세요
  사전: {dictionary}

  질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()

In [ ]:
query

In [ ]:
new_question=dictionary_chain.invoke({"question": query})

new_question

In [78]:
tax_chain = {"query": dictionary_chain} | qa_chain

In [86]:
ai_response = tax_chain.invoke({"question": query})

In [ ]:
ai_response